In [160]:
from urllib.request import Request, urlopen
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd
from time import sleep
import datetime as dt

In [151]:
# URL base de la página que vamos a analizar
base_url = "https://www.logrolimpio.com/catalogo-segunda-vida/?listing-page="

In [152]:
#Creación de la función que extrae el contenido de la web
def extractSoup(url):
    req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    page_soup = soup(webpage, "html.parser")
    return page_soup

def bot_send_text(bot_message):
    bot_token = "7517642163:AAH-QbrmfUYLljOzpDIbVKZFd8wxIN_ffUg"
    bot_chatID = "5544442" #Privado
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&text=' + bot_message
    response = requests.get(send_text)
    return response

def report(producto, categoria, link):
    mensaje = 'Se ha encontrado el producto ' + producto + ' que pertenece a la categoría ' + categoria + '. \n\n' + link
    bot_send_text(mensaje)


In [232]:
productos = []
ids = []
categorias = []
links = []
images = []
fechaActual = str(dt.date.today()- dt.timedelta(days=1))[2:].replace('-','')

In [154]:
def scrapProductos(page_soup):
    #Generar lista con los productos y de ids de la página
    for producto in page_soup.findAll('h3'):
        productos.append(producto.text[:-12])
        ids.append(producto.text[-10:])

def scrapCategorias(page_soup):
    #Extracción de las categorías de los productos
    categoria = page_soup.findAll('li', {"class":"rt-categories"})
    for i in range(len(categoria)):
        categorias.append(categoria[i].get_text(strip=True))
    #return(categorias)

def scrapLinks(page_soup):
    #Extracción de los links de los productos
    for i in page_soup.findAll('h3'):
        for j in i.find_all('a'):
            link = j.get('href')
            links.append(link)

def scrapImagenes(page_soup):
    #Extracción de las urls de las imágenes de los productos
    for listing in page_soup.select('.listing-item .listing-thumb img'):
        img_url = listing.get('src')
        if img_url:
            images.append(img_url)

def scrapTodo(page_soup):
    scrapProductos(page_soup)
    scrapCategorias(page_soup)
    scrapLinks(page_soup)
    scrapImagenes(page_soup)

In [155]:
pagina = 1
while pagina == 1 or page_soup.find('h3') != None:
    url = f"{base_url}{pagina}"
    page_soup = extractSoup(url)
    scrapTodo(page_soup)
    pagina +=1

In [158]:
df = pd.DataFrame({'id': ids,
                   'producto': productos,
                   'categoria': categorias,
                   'link': links,
                   'imagen': images},
                   columns=['id', 'producto', 'categoria', 'link', 'imagen'])

# Guardamos los datos en un archivo CSV
df.to_csv('productos.csv', index=False, encoding='utf-8-sig', sep=';')

In [78]:
for i in range(len(productos)):
    producto = productos[i]
    categoria = categorias[i]
    link = links[i]
    imagen = images[i]
    report(producto, categoria, link)
    sleep(20)

In [241]:
df['producto'][df['id'].str.startswith(fechaActual)]

0        Juego de café de porcelana
1                          Cafetera
2                     Televisor 19″
3                             Trona
4               Patines talla 32-33
5                   Lote de cuadros
6                   Lote de espejos
7           Cartera tejido neopreno
8                    Juego de pesca
9                            Orinal
10                          Frutero
11                           Muñeco
12                  Acuario pequeño
13                 Batidora de vaso
14    Estuche accesorios maquillaje
15                            Bolsa
16            Muñeco sujeta puertas
17                      Monitor 17″
18                Lote de cuchillos
19                             Pony
20         Protectores para enchufe
21                  Lote de cuadros
22              Botella de aluminio
23              Patines talla 28-30
Name: producto, dtype: object

In [77]:
report(producto, categoria, link)

In [235]:
type(mensaje)

NameError: name 'mensaje' is not defined

In [59]:
producto = f'Se ha encontrado el producto {productos[1]}'
print(producto)
type(producto)

Se ha encontrado el producto Maleta de viaje *240830_055


str